In [12]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In [13]:
# log into the virtual coach, update with your credentials
try:
    from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
    vc = VirtualCoach(environment='local')
except ImportError as e:
    print(e)
    print("You have to start this notebook with the command:\
          cle-virtual-coach jupyter notebook")
    raise e

In [14]:
brain_template = '''
# -*- coding: utf-8 -*-
"""
Thimblerigger solver
"""

# pragma: no cover
__author__ = 'Veith Roethlingshoefer'

from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np

n_grid = 3*3
n_out = 3

grid = sim.Population(n_grid, cellclass=sim.IF_curr_exp())
output = sim.Population(n_out, cellclass=sim.IF_curr_exp())
sim.Projection(grid, output, sim.AllToAllConnector(),
               sim.StaticSynapse(weight={syn_weight}))  
circuit = grid + output
'''

In [53]:
grid_to_brain = """
from std_msgs.msg import Int8, UInt8MultiArray
import numpy as np

#@nrp.MapRobotSubscriber("grid_seen", Topic("/thimblerigger_solver/grid_activations", UInt8MultiArray))
@nrp.MapRobotSubscriber('training_signal', Topic("/thimblerigger/training_signal", Int8))
@nrp.MapSpikeSource("grid_neurons", nrp.map_neurons(range(0,9), lambda i: nrp.brain.grid[i]), nrp.fixed_frequency)
@nrp.MapSpikeSource("output", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.output[i]), nrp.fixed_frequency)
@nrp.Robot2Neuron()
def feed_brain(t, training_signal, grid_neurons, output):
    grid = None#grid_seen.value
    training_signal = training_signal.value
    if grid is not None:
        grid = np.fromstring(grid.data, dtype=np.uint8).reshape((grid.layout.dim[0].size, grid.layout.dim[1].size))
        grid = grid.astype(np.float32)
        grid_neurons.rate = grid
    print("test")
    if training_signal is not None:
        training_signal = training_signal.data
        output.rate = 0.
        if training_signal > 0:
            output[training_signal].rate = 10
"""

In [54]:
record_success_csv = """
from std_msgs.msg import Int8
import numpy as np

@nrp.MapCSVRecorder("index_recorder", filename="index_real_estimate.csv",
                    headers=["Time", "real", "estimate"])
@nrp.MapRobotSubscriber('training_signal', Topic("/thimblerigger/training_signal", Int8))
@nrp.MapSpikeSink("estimate", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.actors[i]), nrp.population_rate)
@nrp.Robot2Neuron()
def record_index_csv(t, index_recorder, training_signal, estimate):
    training_signal = training_signal.value
    if training_signal is None:
        return

    training_signal = training_signal.data
    if training_signal > 0:
        index_recorder.record_entry(t,
                                    training_signal,
                                    np.argmax(estimate.rate))
"""

In [55]:
import cv2
import numpy as np
import rospy
import time
from cv_bridge import CvBridge
import sensor_msgs
import std_msgs
from gazebo_msgs.srv import GetModelState, SetModelState
from gazebo_msgs.msg import ModelState
from thread import start_new_thread
import math
import thimblerigger_config as tc
from std_srvs.srv import Trigger, TriggerResponse
from std_msgs.msg import UInt8MultiArray
from std_msgs.msg import MultiArrayDimension

%matplotlib inline

class RobotMover(object):

    def __init__(self, model_name="robot"):
        self.model_name = model_name

    def go_to_pose(self, x, y, orientation):
        pass


class TeleportRobotMover(RobotMover):

    def __init__(self, model_name="robot"):
        self.get_position = rospy.ServiceProxy("/gazebo/get_model_state", GetModelState)
        self.set_position = rospy.ServiceProxy("/gazebo/set_model_state", SetModelState)
        super(TeleportRobotMover, self).__init__(model_name=model_name)

    def go_to_pose(self, x, y, orientation):
        current_robot_pose = self.get_position(self.model_name, "")
        new_state = ModelState()
        new_state.model_name = self.model_name
        new_state.pose = current_robot_pose.pose
        new_state.scale = current_robot_pose.scale
        new_state.twist = current_robot_pose.twist
        new_state.pose.position.x = x
        new_state.pose.position.y = y
        new_state.pose.orientation.x = orientation[1]
        new_state.pose.orientation.y = orientation[2]
        new_state.pose.orientation.z = orientation[3]
        new_state.pose.orientation.w = orientation[0]
        new_state.reference_frame = "world"
        self.set_position(new_state)


class ChallengeInteractor(object):

    def __init__(self):
        rospy.wait_for_service(tc.thimblerigger_show_correct_service)
        print("Show mug service available...")
        rospy.wait_for_service(tc.thimblerigger_hide_correct_service)
        print("Hide mug service available...")
        rospy.wait_for_service(tc.thimblerigger_shuffle_service)
        print("Shuffle service available...")
        print("All thimblerigger services found.")                       
        self.show_correct_mug = rospy.ServiceProxy(tc.thimblerigger_show_correct_service, Trigger)
        self.hide_correct_mug = rospy.ServiceProxy(tc.thimblerigger_hide_correct_service, Trigger)
        self.shuffle = rospy.ServiceProxy(tc.thimblerigger_shuffle_service, Trigger)

                               
class Solver(object):

    def __init__(self):

        self.estimate = [False] * 3
        self.center_points = []
        self.robot_mover = TeleportRobotMover()
        self.current_view = "front"
        self.challenge = ChallengeInteractor()

        self.neuron_grid = None
        self.define_neuron_grid = False
        self.activations = rospy.Publisher('/thimblerigger_solver/grid_activations', UInt8MultiArray, queue_size=5)
        self.send_grid_neuron_activations()  # Set all rates to 0 to list this topic
        self.view = None
        self.set_joints_start()


    def set_joints_start(self):
        neck = rospy.Publisher("/robot/neck_pitch/pos", std_msgs.msg.Float64, queue_size=1)
        l_elbow = rospy.Publisher("/robot/l_elbow/pos", std_msgs.msg.Float64, queue_size=1)
        r_elbow = rospy.Publisher("/robot/r_elbow/pos", std_msgs.msg.Float64, queue_size=1)
        def lower_gaze_and_arms():
            while not rospy.is_shutdown():
                neck.publish(std_msgs.msg.Float64(-0.8))
                l_elbow.publish(std_msgs.msg.Float64(-25.0))
                r_elbow.publish(std_msgs.msg.Float64(-25.0))
            l_elbow.unregister()
            r_elbow.unregister()
        start_new_thread(lower_gaze_and_arms, ())
        time.sleep(2)
        self.side_look()
        time.sleep(2)
        self.view = rospy.Subscriber("/icub_model/left_eye_camera/image_raw", sensor_msgs.msg.Image, self.extract_mugs)
        print("Subscribed to robot view.")
        
    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        print("Closing windows...")
        if self.view is not None:
            self.view.unregister()
        self.activations.unregister()
        # Apparently closing the cv2 windows crashes the jupyter kernel
        #cv2.destroyWindow("Robot extracted view")
        #cv2.waitKey(1)
        print("destroying the solver...")


    def front_look(self):
        self.current_view = "front"
        self.robot_mover.go_to_pose(x=-0.75, y=0., orientation=(0, 0, 1, 0))

    def side_look(self):
        self.current_view = "side"
        orientation = (math.sqrt(0.5), 0, 0, -math.sqrt(0.5))
        self.robot_mover.go_to_pose(x=0.4, y=-0.9, orientation=orientation)

    def allow_define_neuron_grid(self):
        self.define_neuron_grid = True

    def find_neuron_grid(self, contours):
        if self.center_points is None or self.center_points.shape != (3, 2):
            return
        
        x_intervals = [int(self.center_points[0, 0] + 0.5 * \
                        (self.center_points[1, 0] - self.center_points[0, 0])),
                       int(self.center_points[1, 0] + 0.5 * \
                        (self.center_points[2, 0] - self.center_points[1, 0]))]
        y_intervals = []
        for cnt in contours:
            (_,y),radius = cv2.minEnclosingCircle(cnt)
            y_intervals.extend([int(y + 1.5 * radius), int(y - 1.5 * radius)])
        y_intervals = [min(y_intervals), max(y_intervals)]
        self.neuron_grid = x_intervals, y_intervals
        self.define_neuron_grid = False
        print("Neuron grid defined")


    def send_grid_neuron_activations(self):
        locations = np.zeros((3,3))
        if self.neuron_grid is not None:
            for center in self.center_points:
                x_idx = np.searchsorted(self.neuron_grid[0], center[0])
                y_idx = np.searchsorted(self.neuron_grid[1], center[1])
                locations[y_idx, x_idx] = 1

        mat = UInt8MultiArray()
        mat.layout.dim.append(MultiArrayDimension())
        mat.layout.dim.append(MultiArrayDimension())
        mat.layout.dim[0].label = "height"
        mat.layout.dim[1].label = "width"
        mat.layout.dim[0].size = 3
        mat.layout.dim[1].size = 3
        mat.layout.dim[0].stride = 3*3
        mat.layout.dim[1].stride = 3
        mat.layout.data_offset = 0
        mat.data = list(locations.flatten())
        self.activations.publish(mat)
                


    def extract_mugs(self, img_msg, contour_thresh=50, visualize=True):
        img = CvBridge().imgmsg_to_cv2(img_msg, "bgr8")
        most_vibrant_channel = np.argmax(img, axis=2)
        img[most_vibrant_channel != 2] = 0
        img[img[:,:,2] < 150] = 0
        red = img[:, :, 2]
        _, thresh = cv2.threshold(red, 150, 255, 0)
        kernel = np.ones((5,5), np.uint8)
        thresh = cv2.erode(thresh, kernel,iterations = 1)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = [c for c in contours if cv2.contourArea(c) > contour_thresh]

        moments = [cv2.moments(c) for c in contours]
        centers = sorted([(int(M['m10']/M['m00']), int(M['m01']/M['m00'])) for M in moments])
        self.center_points = np.array(centers)


        if self.define_neuron_grid and self.neuron_grid is None:
            self.find_neuron_grid(contours)

        self.send_grid_neuron_activations()


        if visualize:
            if self.neuron_grid is not None:
                for point_x in self.neuron_grid[0]:
                    cv2.line(img, (point_x, 0), (point_x, img.shape[0]), color=(255, 0, 0))
                for point_y in self.neuron_grid[1]:
                    cv2.line(img, (0, point_y), (img.shape[1], point_y), color=(255, 0, 0))
            #for center in centers:
            #    cv2.circle(img, center, radius=1, color=(255,0,0), thickness=3 )
            #cv2.drawContours(img, contours, -1, (0,255,0), 3)
            cv2.imshow("Robot extracted view", img)
            cv2.waitKey(1)

    def find_correct_mug_beginning(self):
        self.allow_define_neuron_grid()
        self.challenge.show_correct_mug()
        print("checking which mug contains the ball")
        correct_mug_id = np.argmin(np.transpose(self.center_points), axis=1)[1]
        self.estimate[correct_mug_id] = True
        print("Found this mug layout: ", self.estimate)
        self.challenge.hide_correct_mug()
        print("found correct mug beginning.")
        
    def shuffle(self):
        print("Shuffling mugs now")
        self.challenge.shuffle()

    def verify_guess(self):
        self.side_look()
        self.challenge.show_correct_mug()
        correct_mug_id = np.argmin(np.transpose(self.center_points), axis=1)[1]
        self.challenge.hide_correct_mug()
        return self.estimate.index(True) == correct_mug_id

In [56]:
import csv
import tempfile
import time
import os

def save_position_csv(sim, datadir):
    with open(os.path.join(datadir, "index_real_estimate.csv"), 'wb') as f:
        cf = csv.writer(f)
        csv_data = sim.get_csv_data("index_real_estimate.csv")
        cf.writerows(csv_data)
        
    """def make_on_status(sim, datadir):
        def on_status(msg):
            if msg["simulationTime"] > 0. and msg["state"] == 'paused':
                print("Trial terminated - saved CSV in {}".format(datadir))
                save_position_csv(sim, datadir)
                sim.stop() 

        return on_status"""

In [58]:
def run_experiment(data_dir, brain_params={'syn_weight': 1.0}):
    sim = vc.launch_experiment('ExDPerceptionChallengeKIT')
    #sim.register_status_callback(make_on_status(sim, data_dir))
    brain_file = brain_template.format(**brain_params)
    sim.edit_brain(brain_file)
    sim.add_transfer_function(record_success_csv)
    sim.start()
    with Solver() as solver:
        sim.add_transfer_function(grid_to_brain)
        solver.find_correct_mug_beginning()
        solver.shuffle()

        print("About to close solver")
        
    sim.pause()
    save_position_csv(sim, data_dir)
    print("Paused simulation, wrote csv file")
    return sim


In [59]:
tmp_folder = tempfile.mkdtemp()
sim = run_experiment(data_dir=tmp_folder)
sim.stop()

Show mug service available...
Hide mug service available...
Shuffle service available...
All thimblerigger services found.
Subscribed to robot view.
checking which mug contains the ball
('Found this mug layout: ', [True, False, False])
found correct mug beginning.
Shuffling mugs now
About to close solver
Closing windows...
destroying the solver...
Paused simulation, wrote csv file


In [60]:
import pandas
csv_file = os.path.join(tmp_folder, "index_real_estimate.csv")
data_frame = pandas.read_csv(csv_file)
score = np.sum(data_frame["real"]==data_frame["estimate"])
print(data_frame)
print(score)
print(score / len(data_frame["real"]))

      Time  real  estimate
0     6.80     2         0
1     6.82     2         0
2     6.84     2         0
3     6.86     2         0
4     6.88     2         0
5     6.90     2         0
6     6.92     2         0
7     6.94     2         0
8     6.96     2         0
9     6.98     2         0
10    7.00     2         0
11    7.02     2         0
12    7.04     2         0
13    7.06     2         0
14    7.08     2         0
15    7.10     2         0
16    7.12     2         0
17    7.14     2         0
18    7.16     2         0
19    7.18     2         0
20    7.20     2         0
21    7.22     2         0
22    7.24     2         0
23    7.26     2         0
24    7.28     2         0
25    7.30     2         0
26    7.32     2         0
27    7.34     2         0
28    7.36     2         0
29    7.38     2         0
..     ...   ...       ...
660  20.00     1         0
661  20.02     1         0
662  20.04     1         0
663  20.06     1         0
664  20.08     1         0
6